In [ ]:
# !pip install -q insightface onnxruntime-gpu deepface opencv-python gradio

In [3]:
import cv2
import numpy as np
import gradio as gr
from deepface import DeepFace
from insightface.app import FaceAnalysis
from PIL import Image

26-01-14 06:11:55 - Directory /root/.deepface has been created
26-01-14 06:11:55 - Directory /root/.deepface/weights has been created


In [4]:
# Cài đặt mô hình
face_app = FaceAnalysis(allowed_modules=['detection'])
face_app.prepare(ctx_id=0, det_size=(640, 640))

download_path: /root/.insightface/models/buffalo_l


100%|██████████| 281857/281857 [00:09<00:00, 31091.37KB/s]


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: /root/.insightface/models/buffalo_l/genderage.onnx genderage
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition
set det-size: (640, 640)


In [5]:
def analyze_face_emotion(input_img):
    if input_img is None:
        return None, "Vui lòng chọn một hình ảnh!"
    img = cv2.cvtColor(np.array(input_img), cv2.COLOR_RGB2BGR)

    faces = face_app.get(img)
    if not faces:
        return None, "Không tìm thấy gương mặt nào trong ảnh."

    largest_face = max(faces, key=lambda f: (f.bbox[2]-f.bbox[0]) * (f.bbox[3]-f.bbox[1]))
    bbox = largest_face.bbox.astype(int)
    x1, y1, x2, y2 = bbox

    padding = 40
    h_img, w_img = img.shape[:2]
    x1, y1 = max(0, x1 - padding), max(0, y1 - padding)
    x2, y2 = min(w_img, x2 + padding), min(h_img, y2 + padding)
    cropped_face = img[y1:y2, x1:x2]

    try:
        results = DeepFace.analyze(img_path = cropped_face,
                                   actions = ['emotion'],
                                   enforce_detection = False)
        res = results[0]
        dominant = res['dominant_emotion']
        emotions = res['emotion']
        cropped_rgb = cv2.cvtColor(cropped_face, cv2.COLOR_BGR2RGB)
        result_text = f"### Cảm xúc chủ đạo: {dominant.upper()}\n\n"
        sorted_emotions = dict(sorted(emotions.items(), key=lambda item: item[1], reverse=True))
        return cropped_rgb, sorted_emotions
    except Exception as e:
        return None, f"Lỗi phân tích: {str(e)}"

In [6]:
# Xây dựng giao diện Web (CSS + HTML tích hợp sẵn trong Gradio)
with gr.Blocks(theme=gr.themes.Soft(), title="AI Emotion Detector") as demo:
    gr.Markdown("""
    # Hệ Thống Phân Tích Cảm Xúc
    Tải ảnh lên để hệ thống tự động nhận diện khuôn mặt và phân tích trạng thái cảm xúc.
    """)
    with gr.Row():
        with gr.Column():
            input_image = gr.Image(label="Ảnh gốc", type="numpy")
            with gr.Row():
                btn_reset = gr.Button("Làm mới", variant="secondary")
                btn_analyze = gr.Button("Phân Tích Cảm Xúc", variant="primary")
        with gr.Column():
            output_face = gr.Image(label="Khuôn mặt đã nhận diện")
            output_chart = gr.Label(label="Tỷ lệ cảm xúc (%)", num_top_classes=5)
    # Xử lý sự kiện
    btn_analyze.click(fn=analyze_face_emotion,
                      inputs=input_image,
                      outputs=[output_face, output_chart])
    btn_reset.click(fn=lambda: [None, None, None],
                    inputs=None,
                    outputs=[input_image, output_face, output_chart])

In [ ]:
# Run web
demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://3abdb69e71d0d124ca.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facial_expression_model_weights.h5
To: /root/.deepface/weights/facial_expression_model_weights.h5


26-01-14 06:14:27 - 🔗 facial_expression_model_weights.h5 will be downloaded from https://github.com/serengil/deepface_models/releases/download/v1.0/facial_expression_model_weights.h5 to /root/.deepface/weights/facial_expression_model_weights.h5...


100%|██████████| 5.98M/5.98M [00:00<00:00, 73.1MB/s]
